# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark 
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ConProj').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',header=True,inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.select('Churn').describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                900|
|   mean|0.16666666666666666|
| stddev| 0.3728852122772358|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [6]:
data.select('Location').describe().show()

+-------+--------------------+
|summary|            Location|
+-------+--------------------+
|  count|                 900|
|   mean|                null|
| stddev|                null|
|    min|00103 Jeffrey Cre...|
|    max|Unit 9800 Box 287...|
+-------+--------------------+



### Let's start with the company column

In [7]:
data.select('Company').distinct().count()

873

In [47]:
by_comp = data.groupBy('Company').count()
by_comp.orderBy(by_comp['count'].desc()).show()

+--------------+-----+
|       Company|count|
+--------------+-----+
|Anderson Group|    4|
|  Williams PLC|    3|
|    Wilson PLC|    3|
|     Evans LLC|    2|
|     Ortiz Ltd|    2|
|     Gates Ltd|    2|
|Smith and Sons|    2|
|      Rice PLC|    2|
|  Williams Ltd|    2|
|Davis and Sons|    2|
|     Smith Ltd|    2|
|      King LLC|    2|
|   Davis Group|    2|
|     Smith Inc|    2|
|    Nelson LLC|    2|
|   Smith Group|    2|
|      Webb PLC|    2|
|      Wise Inc|    2|
|  Williams LLC|    2|
|     Jones LLC|    2|
+--------------+-----+
only showing top 20 rows



As for the column 'Company', it is better to drop it than hold it. The amount of companies are 873, and each one are shown from 1 to 4 times.

In [9]:
data.select('Num_Sites').distinct().show()

+---------+
|Num_Sites|
+---------+
|      8.0|
|      7.0|
|      4.0|
|     11.0|
|     14.0|
|      3.0|
|     10.0|
|     13.0|
|      6.0|
|      5.0|
|      9.0|
|     12.0|
+---------+



### Deal with the location

Let's find out if Location column is helpful enough to be in out logistic regression model. It is better to hold only the zipcode from the whole address.

Some zip codes consist of 10 chars. We will hold only the first 5 chars for every zip code. We will achieve that with 2 ways:
1. With a custom function which splits the zip code at '-' and drop the last chars and
2. With the build in function of pyspark, the substr, which takes 2 arguments: The starting point and the last. The remaining string is droped. 

Then, we will join the two tables by a generated id, with the monotonically_increasing_id function and drop the temporary column 'id'.

In [10]:
data.select('Location').head(15)

[Row(Location='10265 Elizabeth Mission Barkerburgh, AK 89518'),
 Row(Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756'),
 Row(Location='1331 Keith Court Alyssahaven, DE 90114'),
 Row(Location='13120 Daniel Mount Angelabury, WY 30645-4695'),
 Row(Location='765 Tricia Row Karenshire, MH 71730'),
 Row(Location='6187 Olson Mountains East Vincentborough, PR 74359'),
 Row(Location='4846 Savannah Road West Justin, IA 87713-3460'),
 Row(Location='25271 Roy Expressway Suite 147 Brownport, FM 59852-6150'),
 Row(Location='3725 Caroline Stravenue South Christineview, MA 82059'),
 Row(Location='363 Sandra Lodge Suite 144 South Ann, WI 51655-7561'),
 Row(Location='Unit 8120 Box 9160 DPO AA 43432'),
 Row(Location='Unit 1895 Box 0949 DPO AA 40249'),
 Row(Location='897 Kelley Overpass Suite 349 West Rebekahport, AZ 44793'),
 Row(Location='11488 Weaver Cape Hernandezberg, WI 63417-8544'),
 Row(Location='1774 Peter Row Apt. 712 New Autumn, MT 18782')]

In [11]:
from pyspark.sql.window import Window as W
from pyspark.sql.functions import split, monotonically_increasing_id, isnull, max, row_number

In [12]:
zip_code = [word[0][-1] for word in data.select(split('Location', ' ')).collect()]
zip_code[:5]

['89518', '17756', '90114', '30645-4695', '71730']

In [13]:
def del_minus(lst):
    without_minus = []
    for code in lst:
        if '-' in code:
            without_minus.append(code.split('-')[0])
        else:
            without_minus.append(code)
    return without_minus

In [14]:
no_minus_zip = del_minus(zip_code)

In [15]:
no_minus_zip[:5]

['89518', '17756', '90114', '30645', '71730']

In [16]:
zip_code[:5]

['89518', '17756', '90114', '30645-4695', '71730']

In [17]:
data.select('Location', split('Location', ',')[1]).show()

+--------------------+---------------------+
|            Location|split(Location, ,)[1]|
+--------------------+---------------------+
|10265 Elizabeth M...|             AK 89518|
|6157 Frank Garden...|             RI 17756|
|1331 Keith Court ...|             DE 90114|
|13120 Daniel Moun...|        WY 30645-4695|
|765 Tricia Row Ka...|             MH 71730|
|6187 Olson Mounta...|             PR 74359|
|4846 Savannah Roa...|        IA 87713-3460|
|25271 Roy Express...|        FM 59852-6150|
|3725 Caroline Str...|             MA 82059|
|363 Sandra Lodge ...|        WI 51655-7561|
|Unit 8120 Box 916...|                 null|
|Unit 1895 Box 094...|                 null|
|897 Kelley Overpa...|             AZ 44793|
|11488 Weaver Cape...|        WI 63417-8544|
|1774 Peter Row Ap...|             MT 18782|
|45408 David Path ...|        HI 54903-6698|
|28216 Wright Moun...|        DE 40999-2369|
|Unit 4948 Box 481...|                 null|
|69203 Crosby Divi...|             CO 87064|
|9569 Cald

In [18]:
no_minus_zip = spark.createDataFrame([(value,) for value in no_minus_zip], ['no_minus_zip'])

In [19]:
type(no_minus_zip['no_minus_zip'])

pyspark.sql.column.Column

In [20]:
zip_code[0]

'89518'

In [21]:
zip_code[:10]

['89518',
 '17756',
 '90114',
 '30645-4695',
 '71730',
 '74359',
 '87713-3460',
 '59852-6150',
 '82059',
 '51655-7561']

In [22]:
zip_code = spark.createDataFrame([(value,) for value in zip_code], ['zip_code'])

In [23]:
zip_code = zip_code.withColumn('zip', zip_code['zip_code'].substr(1,5)).drop('zip_code')

In [24]:
zip_code = zip_code.withColumn('zip', zip_code['zip'].cast('int'))

In [46]:
zip_code.head(10)

[Row(zip=89518, idx=1),
 Row(zip=17756, idx=2),
 Row(zip=90114, idx=3),
 Row(zip=30645, idx=4),
 Row(zip=71730, idx=5),
 Row(zip=74359, idx=6),
 Row(zip=87713, idx=7),
 Row(zip=59852, idx=8),
 Row(zip=82059, idx=9),
 Row(zip=51655, idx=10)]

In [34]:
zip_code.describe().show()

+-------+------------------+------------------+
|summary|               zip|               idx|
+-------+------------------+------------------+
|  count|               900|               900|
|   mean|50041.432222222225|             450.5|
| stddev|28480.741571480794|259.95191863111916|
|    min|               175|                 1|
|    max|             99942|               900|
+-------+------------------+------------------+



In [37]:
data.select('Churn').describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                900|
|   mean|0.16666666666666666|
| stddev| 0.3728852122772358|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [28]:
zip_code = zip_code.withColumn('idx', monotonically_increasing_id())
data = data.withColumn('idx', monotonically_increasing_id())

In [33]:
zip_code.select(max('idx')).show()

+--------+
|max(idx)|
+--------+
|     900|
+--------+



In [30]:
windowSpec = W.orderBy('idx')

In [31]:
zip_code = zip_code.withColumn('idx', row_number().over(windowSpec))

In [32]:
data = data.withColumn('idx', row_number().over(windowSpec))

In [35]:
data = data.join(zip_code, on='idx', how='inner').drop('idx')

In [36]:
data.head().asDict()

{'Names': 'Cameron Williams',
 'Age': 42.0,
 'Total_Purchase': 11066.8,
 'Account_Manager': 0,
 'Years': 7.22,
 'Num_Sites': 8.0,
 'Onboard_date': datetime.datetime(2013, 8, 30, 7, 0, 40),
 'Location': '10265 Elizabeth Mission Barkerburgh, AK 89518',
 'Company': 'Harvey LLC',
 'Churn': 1,
 'zip': 89518}

In [45]:
data.stat.corr('Churn', 'Age')

0.08592553706723392

In [43]:
data.stat.corr('Churn', 'zip')

0.013873604684394627

### Let's deal with the on board date column